In [1]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

google_notes_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv'
google_definition_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv'

notes = pd.read_csv(google_notes_csv).fillna('')
notes = notes[['Category','Categorization','Word','Definition']].copy()

definitions = pd.read_csv(google_definition_csv).fillna('')

ml_def = definitions.copy()
ml_notes = notes[notes['Category'].isin(['Machine Learning','Visualizations'])]

#from df_processing import notes_df_to_outline_html,final_dataset_for_markdown
#d_learning_notes = final_dataset_for_markdown(notes,definitions)
#d_learning_notes

In [ ]:
# DQ Rules
# No Duplication in definition.Word

In [23]:
def final_dataset_for_markdown_v2(notes=None,
                               definitions=None,
                               export_location='/Users/derekdewald/Documents/Python/Github_Repo/Streamlit/DataDictionary/'):
    
    '''
    
    Function which helps to combined Notes and Definitions into a Single Combined Representation which can ultimately be used as a Learning Reference Tools.

    How this function Works:
    It takes the two sheets and attemps to Consolidate them together to make Final Dataset, generated as d_learning_notes.csv.

    Step 1: Insert Definition for Values in Column CATEGORY has a definition record in column WORD
            notes.Category = definitions.Word
    Step 2: Insert Definition for Values in Column Category where the Corresponding Note
            notes.Category = definition.Category and definition.Categorization = Definition
    
    Step 3: Insert Definitions for Value in Categorization which have definitions in Word Column
        notes.categorization = definitions.word (Add Definition Categorization and Existing Category)
   
    Step 4: Insert Parameter Mapping, where List Type Options Described in Definitions.
           notes.Categorization = definitions.Category and notes.Word = definitions.Categorization

    Order Follows what is in Notes. Uses filtering to rank, and places values with Categorization of Definition for each topic at the top.

    Parameters:
        notes(df): DataFrame of D Notes as stored in: https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv
        definitions(df): DataFrame of D Definitions as stored in: https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv
        export_location(str): Location of where to Save CSV File. If blank, no CSV is made.

    date_created:20-Dec-25
    date_last_modified: 21-Dec-25
    classification:TBD
    sub_classification:TBD
    usage:
        final_dataset_for_markdown()

    ##############

    Has been tested for a Single Value - Machine Learning. Need to Validate once extending.

    ##############

    
    '''

    from data_d_dicts import links

    try:
        len(notes)
    except:
        notes = pd.read_csv(links['google_notes_csv'])

    try:
        len(definitions)
    except:
        definitions = pd.read_csv(links['google_definition_csv'])
    
    temp_def = definitions[['Category','Categorization','Word','Definition']].copy()

    # Step 1
    # Create Unique Category DF
    unique_cat_df = notes.drop_duplicates(['Category'])[['Category']]

    # Process temp_def such that 
    step1_df = unique_cat_df.merge(temp_def.drop('Category',axis=1),left_on='Category',right_on='Word',how='left')
    step1_df['Categorization'] = 'Definition'
    step1_df['Word'] = step1_df['Category'].copy()
    step1_df['Definition'] = step1_df['Definition'].fillna('Not Defined')

    # Step 2
    unique_cat_df['Categorization'] = 'Definition'
    step2_df = unique_cat_df.merge(temp_def,on=['Category','Categorization'],how='inner')

    # Step 3
    step_3df = notes.drop_duplicates('Categorization')[['Category','Categorization']].merge(definitions[['Word','Definition']].rename(columns={'Word':'Categorization'}),on='Categorization',how='left')
    step_3df['Word'] = 'Definition'
    step_3df['Definition'] = temp_df3['Definition'].fillna('Not Defined')
    
    # Step 4
    mod_def = definitions[['Category','Categorization','Word','Definition']].copy()
    mod_def['Definition'] = mod_def['Word'] + ": " + mod_def['Definition']
    mod_def = mod_def.drop('Word',axis=1).reset_index(drop=True)
    mod_def = mod_def.rename(columns={'Categorization':'Word','Category':'Categorization'})
    
    # Merge Notes.Categorization and Notes.Word to Definitions.Category and Definitions.Categorization
    step_4df = notes.drop_duplicates(['Categorization','Word'])[['Category','Categorization','Word']].merge(mod_def,on=['Categorization','Word'],how='inner')
    
    # Can DO ORDER AT THE END. USING THE ORIGINAL DATASET and Category MERGE IN

    final_df = pd.concat([notes,step1_df,step2_df,step_3df,step_4df]).drop_duplicates()

    rank_df1 = notes.drop_duplicates('Category')[['Category']].reset_index(drop=True).reset_index().rename(columns={'index':"Rank1"})
    rank_df2 = notes.drop_duplicates(['Category','Categorization'])[['Category','Categorization']].reset_index(drop=True).reset_index().rename(columns={'index':"Rank2"})
    rank_df2['Rank2'] = rank_df2['Rank2'] + 1
    rank_df3 = notes.drop_duplicates(['Category','Categorization','Word']).reset_index(drop=True).reset_index().drop('Definition',axis=1).rename(columns={'index':'Rank3'})
    rank_df3['Rank3'] = rank_df3['Rank3'] + 1
    final_df = final_df.merge(rank_df1,on=['Category'],how='left').merge(rank_df2,on=['Category','Categorization'],how='left').merge(rank_df3,on=['Category','Categorization','Word'],how='left').fillna(0).sort_values(['Rank1','Rank2','Rank3']).reset_index(drop=True).drop(['Rank1','Rank2','Rank3'],axis=1)

    final_df['COUNT'] = final_df.groupby(['Category','Categorization','Word']).transform('size')
    final_df = final_df[~((final_df['Definition']=='')&(final_df['COUNT']!=1))].drop('COUNT',axis=1).reset_index(drop=True)
    # Need to Delete

    if export_location:
        final_df.to_csv(f"{export_location}d_learning_notes.csv",index=False)
    
    return final_df

d_learning_notes = final_dataset_for_markdown_v2(notes,definitions)  
d_learning_notes

#d_learning_notes = final_dataset_for_markdown(notes,definitions)


,Category,Categorization,Word,Definition
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
1,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...
2,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...
3,Machine Learning,Definition,Bias,Bias refers to the systematic error in a model...
4,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...
...,...,...,...,...
369,Mathematics,Best Linear Unbiased Estimator,Requirements,Linearity: Relationship between Independent an...
370,Mathematics,Best Linear Unbiased Estimator,Requirements,Independence.: Observations should be independ...
371,Mathematics,Best Linear Unbiased Estimator,Requirements,Homoscedasticity: Variance of the residuals sh...
372,Mathematics,Best Linear Unbiased Estimator,Requirements,Normality of Residuals: Residuals should be no...


In [24]:
def search_df_word(df,word,columns=None,case_sensitive=False,print_=True):
    '''
    
    
    '''

    # If User does not define which columns they wish to search, search all.
    if not columns:
        columns = df.columns.tolist()

    final_dict = {}
    final_df = pd.DataFrame()

    for column in columns:
        match_df = df[df[column].fillna('').str.contains(word,case=case_sensitive)]
        if len(match_df)==0:
            final_dict[column] = 'No Matches'
        else:
            final_word = ""
            for word in match_df[column]:
                final_word += word + ","
        
            final_dict[column] = final_word
            final_df = pd.concat([final_df,match_df])

    if print_:
        try:
            print(final_df)
        except:
            pass
            
    return final_df, final_dict

#final_df,final_dict = search_df_word(d_learning_notes,'regularization')
final_df,final_dict = search_df_word(d_learning_notes,'validation')

    

             Category Categorization                   Word  \
307  Machine Learning     Validation             Definition   
308  Machine Learning     Validation                   Goal   
309  Machine Learning     Validation               Approach   
310  Machine Learning     Validation  Important to Remember   
311  Machine Learning     Validation          Lesson Learnt   
312  Machine Learning     Validation              Algorithm   
313  Machine Learning     Validation               Function   
314  Machine Learning     Validation              Procedure   
315  Machine Learning     Validation              Procedure   
316  Machine Learning     Validation              Procedure   
317  Machine Learning     Validation                    TBD   
5    Machine Learning     Definition       Cross Validation   
315  Machine Learning     Validation              Procedure   
316  Machine Learning     Validation              Procedure   

                                            Definition

### Examples of How Information is Included

In [87]:
def d_learning_notes_guide(example_word_dict={},
                           df=d_learning_notes,
                           notes=notes,
                           definitions=definitions):

    if len(example_word_dict)==0:
        example_word_dict= {'Category':'Machine Learning',
                            'Categorization':'Model',
                            'Word':'Bias - Variance Trade Off',
                            'Definition':'Linear Regression'}
    
    for key,word in example_word_dict.items():
        print('\n############################### New Word ###############################\n')
        print(f'Word: {word}')
        print(f'Word Classification: {key}')
    
        for key,data_set in {'notes':notes,'definitions':definitions,'d_learning_notes':d_learning_notes}.items():
    
            print(f"\nInclusion in {key}:")
            for column in ['Category','Categorization','Word','Definition']:
                try:
                    temp = data_set[data_set[column]==word][['Category','Categorization','Word','Definition']].copy()
                except:
                    temp = data_set[data_set[column]==word].copy()
                
                print(f'\nAs Word in {column}')
                
                if len(temp)>0:    
                    display(temp.head(10))
                else:
                    print(f'No Records Found')
                
d_learning_notes_guide()


############################### New Word ###############################

Word: Machine Learning
Word Classification: Category

Inclusion in notes:

As Word in Category


,Category,Categorization,Word,Definition
0,Machine Learning,Problem Definition,Goal,Need this to Pull from Somewhere Else.
1,Machine Learning,Problem Definition,Approach,
2,Machine Learning,Problem Definition,Important to Remeber,
3,Machine Learning,Problem Definition,Lesson Learnt,
4,Machine Learning,Problem Definition,Algorithm,
5,Machine Learning,Problem Definition,Function,
6,Machine Learning,Problem Definition,Procedure,
7,Machine Learning,Problem Definition,TBD,
8,Machine Learning,Data Collection,Goal,Need this to Pull from Somewhere Else.
9,Machine Learning,Data Collection,Approach,



As Word in Categorization
No Records Found

As Word in Word
No Records Found

As Word in Definition
No Records Found

Inclusion in definitions:

As Word in Category


,Category,Categorization,Word,Definition
0,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...
1,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...
2,Machine Learning,Definition,Bias,Bias refers to the systematic error in a model...
3,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...
4,Machine Learning,Definition,Cross Validation,Model evaluation technique used to assess how ...
5,Machine Learning,Definition,Curse of Dimensionality,"As dimensionality grows, data points become in..."
6,Machine Learning,Definition,Deep Learning,Deep learning is a subset of machine learning ...
7,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
8,Machine Learning,Definition,Optimization,Optimization is the process of finding the bes...
9,Machine Learning,Definition,Reinforcement Learning,Reinforcement learning trains an agent to make...



As Word in Categorization
No Records Found

As Word in Word


...


As Word in Definition
No Records Found

Inclusion in d_learning_notes:

As Word in Category
No Records Found

As Word in Categorization
No Records Found

As Word in Word


,Category,Categorization,Word,Definition
4,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...



As Word in Definition
No Records Found

############################### New Word ###############################

Word: Linear Regression
Word Classification: Definition

Inclusion in notes:

As Word in Category
No Records Found

As Word in Categorization
No Records Found

As Word in Word
No Records Found

As Word in Definition
No Records Found

Inclusion in definitions:

As Word in Category
No Records Found

As Word in Categorization
No Records Found

As Word in Word


,Category,Categorization,Word,Definition
109,Model,Algorithm,Linear Regression,Models the relationship between an independent...



As Word in Definition
No Records Found

Inclusion in d_learning_notes:

As Word in Category
No Records Found

As Word in Categorization
No Records Found

As Word in Word
No Records Found

As Word in Definition
No Records Found
